In [ ]:
import os
import tweepy 
import pandas as pd
import csv
import string
import numpy as np
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
from textblob import TextBlob
from gensim.models.phrases import Phrases, Phraser
import re
import multiprocessing
from time import time 

In [ ]:
from nltk.stem import WordNetLemmatizer 
# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Code to fetch tweets


In [ ]:

consumer_key= 'Zm88MmZ4UYJ1o1SqZEUl5blzl'
consumer_secret= '4u0TPzbqEoWt7xoH4mB0ttZyryPRZtKrSHqqvg4xG3r0PLVsm1'
access_token= '1363507078325747721-TiUTfvsD0TfDXzwO0u8HqG0kcxQAEz'
access_token_secret= 'rSGdxdY1d5LokxQUo4xmzvMGhOV8BslNiZGry9DanLpl3'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Open/Create a file to append data
csvFile = open('farmersprotest.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

q=''
ids = set()
for tweet in tweepy.Cursor(api.search, 
                    q="#FarmersProtest", 
                    Since="2021-01-01", 
                    tweet_mode='extended',
                    #until="2014-02-15", 
                    lang="en").items(1000):
    if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
     #Write a row to the csv file/ I use encode utf-8
        csvWriter.writerow([tweet.created_at, tweet.full_text, tweet.favorite_count, tweet.retweet_count, tweet.id])
     #print "...%s tweets downloaded so far" % (len(tweet.id))
        ids.add(tweet.id) # add new id
        print ("number of unique ids seen so far: {}",format(len(ids)))
csvFile.close()

In [ ]:
df=pd.read_csv('farmersprotest_final.csv',header=None,names=['created_at'\
                                                                                             ,'text'\
                                                                                             ,'favorite_count'\
                                                                                            ,'retweet_count'\
                                                                                            ,'id'\
                                                                                            ,'screen_name'])

In [ ]:
df

## Data cleaning

In [ ]:
#convert the text to lowercase
df['preprocessed']=df['text'].str.lower().str.replace('#farmersprotest'," ")

In [ ]:
#function to remove a given pattern
def remove_pattern(input_text,pattern):
    p=re.findall(pattern,input_text)
    for i in p:
        input_text=re.sub(i,"",input_text)
    return input_text

In [ ]:
df['preprocessed']=df['preprocessed'].apply(lambda x: remove_pattern(x,"@[\w]*"))
#Removing Usernames
# clean_text=remove_pattern(remove_spaces,"@[\w]*")

In [ ]:
#removes punctuations,hyperlinks
df['preprocessed']=df['preprocessed'].apply(lambda x: " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", x).split()))


In [ ]:
#Removing white spaces
df['preprocessed']=df['preprocessed'].apply(lambda x: re.sub('/s+'," ",x))

In [ ]:
#function to remove stopwords
def removeStopWords(x):
    words=nltk.tokenize.word_tokenize(x)
#     words_new=[i for i in words if i not in stop+['r']]
    words_new=[i for i in words if i not in ['r']]

    return words_new

In [ ]:
#replacing the most common hashtags
df['preprocessed']=df['preprocessed'].str.replace('shaheedjawan',"")

df['preprocessed']=df['preprocessed'].str.replace('shaheedkisan',"")

df['preprocessed']=df['preprocessed'].str.replace('shaheed',"")

In [ ]:
df['preprocessed']=df['preprocessed'].apply(lambda x: re.sub('/s+'," ",x))

In [ ]:
df['preprocessed']=df['preprocessed'].apply(lambda x: re.sub("^\d+\s|\s\d+\s|\s\d+$", " ",x))

In [ ]:
#re,ove stop words
df['token_words']=df['preprocessed'].apply(removeStopWords)


In [ ]:
#lemmatization
df['lemmatized_words']=df['token_words'].apply(lambda x:[ lemmatizer.lemmatize(text) for text in x])

In [ ]:
df['lemmatized_words']=df['lemmatized_words'].apply(lambda x:[i for i in x if(len(i) >1) ])

In [ ]:
df['lemmatized_sentence']=df['lemmatized_words'].apply(lambda x:' '.join(x))

# Method1 -TextBlob

In [ ]:
#obtaining the polarity
df['polarity']=df['lemmatized_sentence'].apply(lambda x:TextBlob(x).sentiment.polarity)

In [ ]:
#obtaining the subjectivity
df['subjectivity']=df['lemmatized_sentence'].apply(lambda x:TextBlob(x).sentiment.subjectivity)

In [ ]:
df['subject_polar']=(df['subjectivity']+0.01)*df['polarity']# adding 0.01 in order to avoid subjectivity zero

In [ ]:
#function to assign classes
def sentiment_classification(x):
    if(x<0):
        return 'Negative'
    elif(x>0):
        return 'Positive'
    else:
        return 'Neutral'

In [ ]:
df['sentiment_class']=df['polarity'].apply(sentiment_classification)

In [ ]:
df['sentiment_class_new']=df['subject_polar'].apply(sentiment_classification)

In [ ]:
#result
df['sentiment_class'].value_counts(normalize=True)

In [ ]:
df['sentiment_class_new'].value_counts(normalize=True)

##  Method 2 - Word2vec and K-means

In [ ]:
sent = [row for row in df.lemmatized_words]

In [ ]:
df['lemmatized_words']=df['lemmatized_words'].apply(lambda x:[i for i in x if(len(i) >1) ])

## Getting the bigrams

In [ ]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

In [ ]:
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

In [ ]:
#min_count=Ignore all words and bigrams with total collected count lower than this value.


In [ ]:
phrases=Phrases(sent, min_count=2, threshold=1)

In [ ]:
#Bigrams
bigram = Phraser(phrases)
sentences = bigram[sent]#no difference

In [ ]:
# phrases = Phrases(sent, min_count=1, progress_per=50000)


In [ ]:
len(sentences)

##  Word 2 vec

In [ ]:
# w2v_model = Word2Vec(min_count=1,
#                      seed=1,
#                      workers=multiprocessing.cpu_count()-1)
# w2v_model.build_vocab(sentences)


In [ ]:
#initializing the model and building vocabulary
start = time()
w2v_model = Word2Vec(min_count=1,
                     window=2,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     workers=multiprocessing.cpu_count()-1,
                       seed=1
                    ,sg=0)

w2v_model.build_vocab(sentences, progress_per=50000)
print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

In [ ]:
#Training the word2vec model
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

# w2v_model.init_sims(replace=True)

In [ ]:
word_vectors=w2v_model.wv

In [ ]:
vocab_len = len(w2v_model.wv)
vocab_len

In [ ]:
# w2v_model.save("word2vec.model")
# word_vectors = Word2Vec.load("word2vec.model").wv


In [ ]:
# word_vectors=w2v_model.wv

In [ ]:
words = pd.DataFrame(word_vectors.key_to_index.keys())


In [ ]:
#Clustering mmodel
from sklearn.cluster import KMeans
model = KMeans(n_clusters=3, max_iter=1000, random_state=True, n_init=100).fit(X=word_vectors.vectors.astype('double'))

In [ ]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=20, restrict_vocab=None)

## 
we are figuring out which cluster is having negative and positive words

In [ ]:
# positive_cluster_center = model.cluster_centers_[1]
# negative_cluster_center = model.cluster_centers_[0]
# neutral_cluster_center=model.cluster_centers_[2]


In [ ]:
words = pd.DataFrame(word_vectors.key_to_index.keys())

In [ ]:
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])

In [ ]:

words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))


In [ ]:
words.cluster =words.cluster.apply(lambda x: x[0])

In [ ]:
words['cluster'].value_counts()

##  Assigning clusters

In [ ]:
def cluster_value_predict(x):
    if(x==0):
        return -1
    elif(x==1):
        return 0
    else:
        return 1

In [ ]:
words['cluster_value']=words['cluster'].apply(cluster_value_predict)

In [ ]:
# words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]


## Getting the sentiment coefficient


In [ ]:
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).mean()), axis=1)


In [ ]:
# words['closeness_score_min'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
# words['closeness_score_max'] = words.apply(lambda x: 1/(model.transform([x.vectors]).max()), axis=1)


In [ ]:
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [ ]:
# words['sentiment_coeff_min'] = words.closeness_score_min * words.cluster_value

In [ ]:
# words['sentiment_coeff_max'] = words.closeness_score_max * words.cluster_value

In [ ]:
words.columns

In [ ]:
words[['words','cluster_value','sentiment_coeff']]

In [ ]:
# words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

## Sentiment dictionary

In [ ]:
sentiment_dict = dict(zip(words['words'], words['sentiment_coeff']))

## TFIDF Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
# tfidf = TfidfVectorizer(norm='l2')


In [ ]:
tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm='l2')
tfidf.fit(df.lemmatized_sentence)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(df.lemmatized_sentence)

In [ ]:
# transformed

In [ ]:
#creating the TFIDF dictionary and replacing the tweets withtfidf scores
def create_tfidf_dictionary(x, transformed_file, features):
 
  
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
   
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.lemmatized_sentence.split()))

In [ ]:
%%time
replaced_tfidf_scores =df.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)
#this step takes around 3-4 minutes minutes to calculate

In [ ]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dictionary
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [ ]:
replaced_closeness_scores = df.lemmatized_sentence.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))


In [ ]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, df.id,df.lemmatized_sentence, df.text]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'id','lemmatized_sentence', 'tweets']


## Dot products of 2 sentence vectors

In [ ]:
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)


##  converting bool (boolean to integer )

In [ ]:
# replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
# replacement_df['sentiment'] = [1 if i==1 else 0 for i in replacement_df.sentiment]

In [ ]:
# replacement_df.tail()

In [ ]:
#Setting the benchmark for each cluster
replacement_df['prediction']=np.where(replacement_df['sentiment_rate']>1,'positive'
                                      ,np.where(replacement_df['sentiment_rate']<-0.5,'negative','neutral'))

In [ ]:
#summary of the output
replacement_df['prediction'].value_counts(normalize=True)

In [ ]:
#function to scale the result
def cluster_value_predict(x):
    if(x>1):
        return 1
    elif(x<-0.4):
        return -1
    else:
        return 0

In [ ]:
replacement_df['sentiment_class']=replacement_df['sentiment_rate'].apply(cluster_value_predict)

In [ ]:
replacement_df.to_csv('final_updated_out.csv',index=False)

In [ ]:
replacement_df.loc[11146]

## Visualization

* comparing textblob to user defined built alogirithm

In [ ]:
defined_algo=replacement_df['prediction'].value_counts().reset_index()


In [ ]:
textblob_algo=df['sentiment_class_new'].value_counts().reset_index()

In [ ]:
import seaborn as sns  
import matplotlib.pyplot as plt  
sns.set(color_codes=True) 

In [ ]:
defined_algo

In [ ]:
textblob_algo['index']=textblob_algo['index'].str.lower()

In [ ]:
combineOutput=defined_algo.merge(textblob_algo,on='index')

In [ ]:
combineOutput.rename(columns={'sentiment_class_new':'TextBlob','prediction':'Word2vec-Kmeans'},inplace=True)

In [ ]:
#plotting the visualisation
combineOutput.plot(x="index", y=["Word2vec-Kmeans", "TextBlob"], kind="bar")


In [ ]:
combineOutput=defined_algo.merge(textblob_algo,on='index')

In [ ]:
pd.concat({
    'One': df1.set_index('Brand').Count, 'Two': df2.set_index('Brand').Count
}, axis=1).plot.bar()

In [ ]:
sns.barplot(x=defined_algo['index'],y=defined_algo['prediction'])